## 東森新聞雲爬蟲練習

練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。

學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。


In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
import time

In [6]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

browser = webdriver.Chrome('./chromedriver.exe')
browser.get(ETtoday_url)

last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:

        # Scroll down to the bottom.
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load the page.
        time.sleep(2)

        # Calculate new scroll height and compare with last scroll height.
        new_height = browser.execute_script("return document.body.scrollHeight")

        if new_height == last_height:

            break

        last_height = new_height

In [7]:
def get_news_detail_content(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find('div', attrs={'class': 'story'}).find_all('p')

    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)


解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。


In [8]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
browser.close()
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    get_news_detail_content(externalLink)


導
隨著新冠肺炎(COVID-19)的疫情增溫，台灣人對於口罩的需求也日益增長；不少民眾問「口罩哪裡買？」讓網路上出現許多口罩數量的搜尋管道，而在LINE官方帳號群組上一夕爆紅的「敗口罩」也因而誕生。《ETtoday新聞雲》記者這次也找到了「敗口罩」開發者陳葵懋，請他分享開發「敗口罩」的過程與當中的一些插曲。
有人笑稱，新冠肺炎在台灣所引起口罩之亂，意外地開啟了全台灣最大的「黑客松」，有許多IT技術人士都紛紛加入了口罩地圖的開發，以提供民眾「哪裡還有口罩」的資訊，而陳葵懋也是其中一員。
從事於商業應用系統軟體設計與開發的陳葵懋，同時也是微軟MVP(最有價值專業人士)，這次的「敗口罩」聊天機器人的開發，他就是利用微軟Azure服務上的Chatbot Application開發平台為基礎，開發出了在LINE上供大家詢問的「敗口罩」。
陳葵懋表示，產生開發「敗口罩」想法的時間點要回推到還在超商搶口罩的時期，當時剛好網路上有人架設了互助回報數量的平台，並且提供資料可以串接，因此就想說來寫個應用方便自已跟朋友查詢。
▼「哪間藥局還有口罩？」已經成為民眾必問的問題 。（圖／《ETtoday新聞雲》資料照）
因此就想說利用直接藉由Chatbot的形態是最簡單快速的，也是因為Line是大家很熟悉的軟體，在台灣用Line通訊的使用者還是比Telegram、Facebook Messenger來的多，開發者只需要著重於資料的串接，不用再花時間設計使用介面，可以用最少的時間，快速撰寫出需要的功能，使用者也不用過多的學習，自然就會操作了。
再者，使用者一開始的目標很明確就是要能查週遭口罩販售點，因此功能盡可能簡單、方便使用為主，過於複雜的使用者介面反而不好用。
「敗口罩」的第一版(超商版)僅花了4小時完成，(從Open Data 資料整理到完成部署到Azure上)，只不過沒想到當時在完成後，當天晚上就宣佈改由健保藥局才能販售，就這樣很「短命」的Linebot只活了12小時。
「原本打算那就這樣了。」陳葵懋表示，後來有朋友私訊，看能不能改為藥局版本，畢竟必須是健保藥局才能販售，不是所有藥局都可以，後來想想也好，就這樣給予了這支Linebot新生命。
▼超商版的「敗口罩」僅僅存活了12小時。（圖／記者邱倢芯攝）
另外，微軟Azure服務的Chatbot Application具有支援跨平台的優